In [30]:
#!/usr/bin/env python3
"""Task 3: Variational Autoencoder"""
import tensorflow.keras as keras


def autoencoder(input_dims, hidden_layers, latent_dims):
    """Creates a variational autoencoder
    Args:
        input_dims (int): Contains the dimensions of the model input
        hidden_layers (list): Contains the number of nodes for each hidden
            layer; the hidden layers should be reversed for the decoder
        latent_dims (int): Contains the dimensions of the latent space
    Returns:
        encoder, decoder, auto (tuple): Contains the encoder, decoder and
            autoencoder models, respectively"""


    # Encoder
    encoder_inputs = keras.Input(shape=(input_dims,))
    x = encoder_inputs

    for units in hidden_layers:
        x = keras.layers.Dense(units, activation='relu')(x)

    # Latent space
    latent_mean = keras.layers.Dense(latent_dims, activation=None)(x)
    latent_log_variance = keras.layers.Dense(latent_dims, activation=None)(x)

    # Reparameterization trick
    def sampling(args):
        latent_mean, latent_log_variance = args
        epsilon = keras.backend.random_normal(shape=(keras.backend.shape(latent_mean)[0], latent_dims),
                                              mean=0.0, stddev=0.1)
        return latent_mean + keras.backend.exp(latent_log_variance) * epsilon

    latent_space = keras.layers.Lambda(sampling)([latent_mean, latent_log_variance])

    # Decoder
    decoder_inputs = keras.Input(shape=(latent_dims,))
    x = decoder_inputs

    for units in reversed(hidden_layers):
        x = keras.layers.Dense(units, activation='relu')(x)

    decoder_outputs = keras.layers.Dense(input_dims, activation='sigmoid')(x)

    # Define encoder and decoder models
    encoder = keras.Model(encoder_inputs, [latent_space, latent_mean, latent_log_variance], name='encoder')
    decoder = keras.Model(decoder_inputs, decoder_outputs, name='decoder')

    # Define full autoencoder model
    autoencoder_outputs = decoder(encoder(encoder_inputs)[0])
    autoencoder = keras.Model(encoder_inputs, autoencoder_outputs, name='autoencoder')

    # Compile the autoencoder model
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return encoder, decoder, autoencoder


In [31]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


np.random.seed(0)
tf.compat.v1.set_random_seed(0)
encoder, decoder, auto = autoencoder(784, [512, 256], 2)
if len(auto.layers) == 3:
    print(auto.layers[0].input_shape == [(None, 784)])
    print(auto.layers[1] is encoder)
    print(auto.layers[2] is decoder)

with open('1-test', 'w+') as f:
    x_test = np.load("/home/ediddev/School Repos/holbertonschool-machine_learning/supervised_learning/data/MNIST.npz")["X_test"]
    x_test = x_test[:256].reshape((-1, 784))
    f.write(np.format_float_scientific(auto.evaluate(x_test, x_test, verbose=False), precision=6) + '\n')
    f.write(auto.optimizer.__class__.__name__ + '\n')

with open('2-test', 'w+') as f:
    try:
        f.write(encoder.layers[0].__class__.__name__ + '\n')
        f.write(str(encoder.layers[0].input_shape) + '\n')
    except:
        f.write('FAIL\n')
    for layer in encoder.layers[1:]:
        try:
            f.write(layer.__class__.__name__ + '\n')
            if layer.__class__.__name__ == 'Dense':
                if layer.activation != None:
                    f.write(layer.activation.__name__ + '\n')
                f.write(str(layer.input_shape) + '\n')
                f.write(str(layer.output_shape) + '\n')
            elif layer.__class__.__name__ == 'Lambda':
                assert(len(layer.input) == 2)
                assert(encoder.layers[-3].output in layer.input)
                assert(encoder.layers[-2].output in layer.input)
                f.write(str(layer.input_shape) + '\n')
                f.write(str(layer.output_shape) + '\n')
        except:
            f.write('FAIL\n')

with open('3-test', 'w+') as f:
    try:
        f.write(decoder.layers[0].__class__.__name__ + '\n')
        f.write(str(decoder.layers[0].input_shape) + '\n')
    except:
        f.write('FAIL\n')
    for layer in decoder.layers[1:]:
        try:
            f.write(layer.__class__.__name__ + '\n')
            if layer.__class__.__name__ == 'Dense' and layer.activation != None:
                f.write(layer.activation.__name__ + '\n')
            f.write(str(layer.input_shape) + '\n')
            f.write(str(layer.output_shape) + '\n')
        except:
            f.write('FAIL\n')

True
True
True
